# Auto-ajuste de hiperparámetros por búsqueda de cuadrícula (grid-search)

## Nuestro modelo predictivo

In [ ]:
from sklearn import set_config

set_config(display="diagram")

In [ ]:
import pandas as pd

adult_census = pd.read_csv("../../data/adult-census-numeric/full.csv")

In [ ]:
# Extraemos la columna que contiene el objetivo.

target_name = "class"
target = adult_census[target_name]
target

In [ ]:
# Quitamos el objetivo de los datos y la columna "Education-Num" (duplicado de "Educación").

data = adult_census.drop(columns=[target_name, "education-num"])
data.head()

In [ ]:
# La dividimos en un conjunto de entrenamiento y prueba.

from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42)

In [ ]:
# Definiremos un piepline. Gestionará características numéricas y categóricas.

# seleccionar todas las columnas categóricas.

from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)

Aquí usaremos un modelo basado en árbol como clasificador (`HistgradientBoostingClassifier`).
Esto significa que:
- las variables numéricas no necesitan escala;
- las variables categóricas se pueden tratar con un entorno ordinal incluso si el orden de codificación no es significativo;
- para los modelos basados ​​en árboles, OrdinalEncoder evita tener representaciones de alta dimensión.

In [ ]:
# Ahora construimos nuestro OrdinalEncoder pasando las categorías conocidas.

from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)

In [ ]:
# usamos un ColumnTransformer para seleccionar las columnas categóricas y aplicar el OrdinalEncoder a ellas.

from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ('cat_preprocessor', categorical_preprocessor, categorical_columns)],
    remainder='passthrough', sparse_threshold=0)

In [ ]:
# Finalmente, usamos un clasificador basado en árbol (histogram gradient-boosting) para predecir si una persona gana o no más de 50 K$ al año.

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", HistGradientBoostingClassifier(random_state=42, max_leaf_nodes=4))])

model

## Ajustar con búsqueda de cuadrícula
- Usamos el **estimador GridSearchCV** para hacer la búsqueda.
    - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- Dado que la búsqueda de la cuadrícula será costosa, solo exploraremos la combinación `learning_rate` y `max_leaf_nodes`.

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

param_grid = {
    'classifier__learning_rate': (0.01, 0.1, 1, 10),
    'classifier__max_leaf_nodes': (3, 10, 31, 50)
}

model_grid_search = GridSearchCV(model, param_grid=param_grid, cv=2)

model_grid_search.fit(data_train,target_train)

In [ ]:
# Finalmente, verificaremos la precisión de nuestro modelo utilizando el conjunto de pruebas.

accuracy = model_grid_search.score(data_test, target_test)

accuracy

In [ ]:
model_grid_search.get_params()

Una vez que se la búsqueda de la cuadrícula está ajustada, se puede usar como cualquier otro predictor llamando a `predict` y `predict_probe`.

Internamente, utilizará el modelo con los mejores parámetros encontrados durante el ajuste.

In [ ]:
# Obtener predicciones para las 5 primeras muestras utilizando el estimador con los mejores parámetros.
model_grid_search.predict(data_test.iloc[0:5])

In [ ]:
# se puede conocer estos parámetros mirando el atributo best_params_.

model_grid_search.best_params_


In [ ]:
# Además, podemos inspeccionar todos los resultados que se almacenan en el atributo cv_results_ de la búsqueda de cuadrícula.
# filtramos algunas columnas específicas de estos resultados.

# model_grid_search.cv_results_

cv_results = pd.DataFrame(model_grid_search.cv_results_).sort_values(
    "mean_test_score", ascending=False)
cv_results

In [ ]:
# Centrémonos en las columnas más interesantes. Acortamos por legibilidad los nombres de los parámetros para eliminar el prefijo "param_classifier__":

# Obtener los nombres de los parámetros
column_results = [f"param_{name}" for name in param_grid.keys()]
column_results += [
    "mean_test_score", "std_test_score", "rank_test_score"]
cv_results = cv_results[column_results]

In [ ]:
def shorten_param(param_name):
    if "__" in param_name:
        return param_name.rsplit("__", 1)[1]
    return param_name


cv_results = cv_results.rename(shorten_param, axis=1)
cv_results

Podemos visualizar la búsqueda de la cuadrícula como un mapa de calor.
Necesitamos transformar nuestros cv_results en un DataFrame, donde:
- Las filas corresponderán a los valores de tasa de aprendizaje;
- Las columnas corresponderán a max_leaf_nodes;
- El contenido del DataFrame será los puntajes medios de prueba.

In [ ]:
pivoted_cv_results = cv_results.pivot_table(
    values="mean_test_score", index=["learning_rate"],
    columns=["max_leaf_nodes"])

pivoted_cv_results

In [ ]:
# mapa de calor
import seaborn as sns

ax = sns.heatmap(pivoted_cv_results, annot=True, cmap="YlGnBu", vmin=0.7,
                 vmax=0.9)
ax.invert_yaxis()

## **Ejercicio**

Para el dataset "house-prices/full.csv" y usando [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html): 
- Autoajusta parametros usando grid search.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Carga de datos
haouses = pd.read_csv("../../data/house-prices/full.csv")
haouses.columns
# haouses.shape

In [ ]:
target_name = "SalePrice"

y = haouses[target_name]
X = haouses.drop(columns=target_name)

In [ ]:
y

In [ ]:
# Mostrar información del dataset
print("Características del dataset:")
print(f"- Número de muestras: {X.shape[0]}")
print(f"- Número de características: {X.shape[1]}")
print(f"\nNombres de las características:\n{X.columns}")
print(f"\nDescripción de las características:\n{X.info()}...")

# Ver primeras filas
X.head()

In [ ]:
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)

In [ ]:
preprocessor = ColumnTransformer(
    [
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ])


In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    X, y, random_state=42)

## Lasso

In [ ]:
lasso_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Lasso(alpha=1.0))
])

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

param_grid = {
    'regressor__alpha': (0.01, 0.1, 1, 10,100),
    'regressor__max_iter': (10, 100, 1000, 10000)
}

# model_grid_search = GridSearchCV(lasso_pipeline, param_grid=param_grid, cv=2) // scoring por defecto: r2
model_grid_search = GridSearchCV(lasso_pipeline, param_grid=param_grid, cv=2, scoring='neg_mean_absolute_error')

model_grid_search.fit(data_train,target_train)

In [ ]:
result = model_grid_search.score(data_test, target_test)
result

In [ ]:
model_grid_search.best_params_

In [ ]:
model_grid_search.cv_results_.keys()

In [ ]:

cv_results = pd.DataFrame(model_grid_search.cv_results_).sort_values(
    "mean_test_score", ascending=False)
cv_results